📘 Module 1: Python Object Model & C-Structures

Мета: Зрозуміти, що відбувається в оперативній пам'яті (RAM), коли ви створюєте змінну, і як виглядає "атом" Python — структура PyObject.

1. Вступ: "Змінних" не існує
У мовах на кшталт C++, змінна — це коробка (container), у якої є фіксована адреса пам'яті, і ви кладете туди дані. У Python змінна — це стікер (label/tag) або вказівник, який ви клеїте на об'єкт, що висить десь у пам'яті.

Коли ви пишете:

In [1]:
a = 1000

В CPython відбувається наступне:
1. Алокатор виділяє пам'ять під об'єкт integer(1000).
2. Створюється ім'я a у просторі імен (namespace).
3. Ім'я a вказує (посилається) на адресу цього об'єкта.

🔬 Практика: Доказ через id()
Функція id() повертає адресу об'єкта в пам'яті (у CPython).

In [2]:
a = 300
b = 300

# Перевіримо, чи це один і той самий об'єкт, чи різні?
print(f"Address of a: {id(a)}")
print(f"Address of b: {id(b)}")

# 'is' перевіряє рівність адрес (id(a) == id(b))
print(f"a is b? {a is b}")

# Assignment (Присвоєння) просто клеїть ще один стікер на той самий об'єкт
c = a
print(f"Address of c: {id(c)}")
print(f"a is c? {a is c}")

Address of a: 2836737635856
Address of b: 2836739003984
a is b? False
Address of c: 2836737635856
a is c? True


Пояснення: У цьому прикладі для 300 Python створив два різних об'єкти (тому адреси різні). Але коли ми зробили c = a, ми не скопіювали число. Ми скопіювали посилання. Тепер і a, і c вказують на одну ділянку пам'яті.

Важливо розрізняти два оператори:
- `==` порівнює **значення** (чи рівні дані?)
- `is` порівнює **ідентичність** (чи це один і той самий об'єкт в пам'яті?)

In [5]:
# Важливо: == перевіряє ЗНАЧЕННЯ, is перевіряє ІДЕНТИЧНІСТЬ (адресу)
a = 300
b = 300

print(f"a == b (values equal)?  {a == b}")   # True — значення однакові
print(f"a is b (same object)?   {a is b}")   # False — різні об'єкти в пам'яті

# Для малих чисел (-5..256) Python кешує об'єкти (Small Integer Cache)
x = 100
y = 100
print(f"\nx == y (values equal)?  {x == y}")   # True
print(f"x is y (same object)?   {x is y}")   # True — один і той самий кешований об'єкт!

a == b (values equal)?  True
a is b (same object)?   False

x == y (values equal)?  True
x is y (same object)?   True


2. Анатомія PyObject (C-Level)
Кожен об'єкт у Python (число, рядок, список, функція) на рівні C починається з одного і того ж заголовка. Це називається PyObject Head.

```
typedef struct _object {
    Py_ssize_t ob_refcnt;   // Reference Count (Лічильник посилань)
    PyTypeObject *ob_type;  // Pointer to Type (Вказівник на тип)
} PyObject;
```

Будь-який об'єкт у пам'яті містить мінімум ці два поля:

1. ob_refcnt: Скільки змінних посилаються на цей об'єкт. Якщо стане 0 — пам'ять звільниться.

2. ob_type: Вказівник на клас об'єкта (наприклад, на клас int або list). Саме звідси Python знає, що це число і як його додавати.

Якщо об'єкт має змінний розмір (наприклад, list або str), він має заголовок PyVarObject:

```
typedef struct {
    PyObject ob_base;     // Ті самі refcnt та type
    Py_ssize_t ob_size;   // Кількість елементів (не байтів, а items!)
} PyVarObject;
```

🔬 Практика: Дивимось "під капот"
Ми можемо побачити кількість посилань (ob_refcnt) за допомогою sys.getrefcount.

In [6]:
import sys

# Створюємо унікальний об'єкт
my_var = object()

# ПИТАННЯ НА СПІВБЕСІДІ:
# Чому getrefcount показує 2, хоча ми створили тільки одну змінну 'my_var'?
print(f"References count: {sys.getrefcount(my_var)}")

# Створимо ще одне посилання
another_tag = my_var
print(f"References count after alias: {sys.getrefcount(my_var)}")

References count: 2
References count after alias: 3


Глибокий аналіз:
Результат перший: 2.

Чому?

Змінна my_var (глобальний простір імен). Та аргумент функції sys.getrefcount(my_var) (тимчасове посилання всередині функції).

Це прямий доказ існування поля ob_refcnt у структурі C.

3. Розмір об'єктів (Overhead)
Через те, що кожен об'єкт тягає за собою C-заголовок (PyObject), навіть прості дані займають багато місця.

У мові C int32 займає 4 байти. А скільки займає int у Python?

In [7]:
import sys

# Звичайне число
x = 10
size_int = sys.getsizeof(x)
print(f"Size of int(10): {size_int} bytes")

# Порожній список
empty_list = []
size_list = sys.getsizeof(empty_list)
print(f"Size of empty list: {size_list} bytes")

Size of int(10): 28 bytes
Size of empty list: 56 bytes


Чому так багато? Для int (28 байт у 64-bit Python):

1. 8 bytes: ob_refcnt
2. 8 bytes: ob_type pointer
3. 8 bytes: ob_size (тому що Python int — це big integer змінної довжини)
4. 4 bytes: Саме значення (цифра 10).
5. Padding/Alignment (вирівнювання пам'яті).

Висновок: Python обмінює пам'ять на зручність і динамічну типізацію.

4. Mutability vs Immutability: Вплив на пам'ять
Це найважливіша концепція для розуміння продуктивності.

Immutable (Незмінні): int, float, str, tuple, frozenset.

Механіка: Ви не можете змінити вміст пам'яті. Будь-яка зміна створює НОВИЙ об'єкт.

Mutable (Змінні): list, dict, set, bytearray.

Механіка: Ви можете змінювати вміст пам'яті за тією ж адресою.

🔬 Практика: Оператор += (In-place addition)
Поведінка += кардинально відрізняється для цих двох типів.

In [8]:
print("--- IMMUTABLE (Tuple) ---")
# Кортеж незмінний
t = (1, 2)
old_id = id(t)
print(f"Old ID: {old_id}")

t += (3,)  # Спроба додати елемент
new_id = id(t)
print(f"New ID: {new_id}")
print(f"Is same object? {old_id == new_id}")
# Висновок: Створено НОВИЙ об'єкт, змінна 't' переклеєна на нього.

print("\n--- MUTABLE (List) ---")
# Список змінний
l = [1, 2]
old_id = id(l)
print(f"Old ID: {old_id}")

l += [3]  # Внутрішня зміна (in-place)
new_id = id(l)
print(f"New ID: {new_id}")
print(f"Is same object? {old_id == new_id}")
# Висновок: Об'єкт той самий. Ми розширили виділену пам'ять.

--- IMMUTABLE (Tuple) ---
Old ID: 2836738707008
New ID: 2836739352800
Is same object? False

--- MUTABLE (List) ---
Old ID: 2836738826816
New ID: 2836738826816
Is same object? True


🧠 Mind-Bending Puzzle

Що станеться тут? Ми маємо кортеж (незмінний), який містить список (змінний).

In [9]:
weird_tuple = ([1, 2], 3)

try:
    # Спробуємо додати елемент до списку всередині кортежу через +=
    # += викликає list.__iadd__([3]), який модифікує список in-place і повертає self,
    # а потім намагається зробити tuple[0] = result — це і дає TypeError.
    weird_tuple[0] += [3]
except TypeError as e:
    print(f"Caught Error: {e}")

# Але чи змінився список?
print(f"Tuple content: {weird_tuple}")

Caught Error: 'tuple' object does not support item assignment
Tuple content: ([1, 2, 3], 3)


Результат:

Ви отримаєте помилку TypeError, тому що кортеж забороняє присвоювання (tuple does not support item assignment).

АЛЕ список всередині зміниться! Ви побачите ([1, 2, 3], 3).

Чому? Операція `+=` на списку використовує `__iadd__` (не `__add__`!), і працює в два етапи:

1. **`list.__iadd__([3])`** — список модифікує себе in-place (extend), повертає `self`. Це **вже відбулося** успішно.
2. **Assignment: `tuple[0] = result`** — Python намагається записати результат назад у кортеж. Кортеж каже "Ні" і кидає помилку. Але список вже змінений!

Важливо: це саме `__iadd__`, а не `__add__`. `__add__` створив би новий список, і оригінал не змінився б.

In [10]:
weird_tuple = ([1, 2], 3)

try:
    # Спробуємо додати елемент до списку всередині кортежу через +=
    # Це технічно: weird_tuple[0] = weird_tuple[0] + [3]
    weird_tuple[0].extend([3])
except TypeError as e:
    print(f"Caught Error: {e}")

# Але чи змінився список?
print(f"Tuple content: {weird_tuple}")

Tuple content: ([1, 2, 3], 3)


Ні, цей код не видасть помилку.Це дуже важливий нюанс, який відрізняє роботу методів від операторів присвоєння.Чому помилки не буде?У попередньому прикладі ми використовували оператор +=.Оператор += у Python працює так: a += b -> a = a.__iadd__(b).Тобто він намагається перезаписати значення в комірку кортежу (weird_tuple[0] = ...), навіть якщо саме посилання не змінилося. Кортеж (tuple) це забороняє, тому виникає TypeError.У твоєму новому коді ти використовуєш метод .extend(): weird_tuple[0].extend([3])

Тут відбувається наступне:
Python бере об'єкт за індексом 0 (це наш список).
Він викликає у цього об'єкта метод .extend().
Список змінюється всередині себе (in-place).
Ніякого присвоєння (=) назад у кортеж не відбувається.

Кортежу байдуже, що відбувається всередині об'єкта, на який він посилається. Йому важливо лише те, щоб саме посилання (адреса пам'яті, яку він зберігає) залишалося незмінним. Оскільки .extend() не змінює адресу списку і не намагається перезаписати її в кортеж, все працює ідеально.

Висновок для Senior розробника:

weird_tuple[0] += [3] — Помилка (бо є спроба присвоєння: tuple[0] = ...).

weird_tuple[0].extend([3]) — Ок (бо це чистий виклик методу, посилання в кортежі не чіпаються).